### Delta Python API

In [0]:
from delta.tables import DeltaTable

#### Delta Table Creation

In [0]:
DeltaTable.createOrReplace(spark) \
  .tableName("my_catalog.my_schema.delta_four") \
  .addColumn("id", "int")\
  .addColumn("name", "string")\
  .execute()

#### Computed Column

In [0]:
DeltaTable.createOrReplace(spark) \
  .tableName("my_catalog.my_schema.delta_four") \
  .addColumn("id", "int")\
  .addColumn("name", "string")\
  .addColumn("salary", "double")\
  .addColumn("salaryAfterTax", "double", generatedAlwaysAs="salary*0.90")\
  .execute()

In [0]:
%sql
insert into my_catalog.my_schema.delta_four (id, name, salary)
values
(4, 'Mary', 100000),
(5, 'Mike', 120000)

num_affected_rows,num_inserted_rows
2,2


In [0]:
%sql
select * from my_catalog.my_schema.delta_four;

id,name,salary,salaryAfterTax
1,John,100000.0,90000.0
2,Jane,120000.0,108000.0
3,Joe,150000.0,135000.0


#### Write Delta to Volume

In [0]:
df = spark.read.table("my_catalog.my_schema.delta_four")

df.write.format("delta")\
    .option("path", "/Volumes/my_catalog/my_schema/my_volume/delta_four/")\
    .save()

In [0]:
%sql
insert into delta.`/Volumes/my_catalog/my_schema/my_volume/delta_four/` (id, name, salary)
values
(4, 'Mary', 100000),
(5, 'Mike', 120000);

num_affected_rows,num_inserted_rows
2,2


In [0]:
%sql
select * from delta.`/Volumes/my_catalog/my_schema/my_volume/delta_four/`;

id,name,salary,salaryAfterTax
1,John,100000.0,90000.0
2,Jane,120000.0,108000.0
3,Joe,150000.0,135000.0
4,Mary,100000.0,null
5,Mike,120000.0,null


#### DML

In [0]:
%sql
update delta.`/Volumes/my_catalog/my_schema/my_volume/delta_four/`
set salary = salary * 0.70
where id = 2;

num_affected_rows
1


In [0]:
spark.read.format("json")\
      .load("/Volumes/my_catalog/my_schema/my_volume/delta_four/_delta_log/00000000000000000008.json").display()

commitInfo,metaData,protocol
"List(0815-125944-m043vv0y-v2n, Databricks-Runtime/17.0.x-aarch64-photon-scala2.13, false, Serializable, RESTORE, List(1, 0, 0, 0, 0, 1401), List(null, 6), 7, 1755264877092, 9689b3ee-120d-4c1b-82ad-e5d68e822a1c, 78362573187593, phyominnthwin@gmail.com)",null,null
null,"List(List(4, name, true), 1755263377467, List(parquet), 66814450-ee69-4751-9f86-07d4ba349df6, List(), {""type"":""struct"",""fields"":[{""name"":""id"",""type"":""integer"",""nullable"":true,""metadata"":{""delta.columnMapping.id"":1,""delta.columnMapping.physicalName"":""id""}},{""name"":""name"",""type"":""string"",""nullable"":true,""metadata"":{""delta.columnMapping.id"":2,""delta.columnMapping.physicalName"":""name""}},{""name"":""salary"",""type"":""double"",""nullable"":true,""metadata"":{""delta.columnMapping.id"":3,""delta.columnMapping.physicalName"":""salary""}},{""name"":""salaryAfterTax"",""type"":""double"",""nullable"":true,""metadata"":{""delta.columnMapping.id"":4,""delta.columnMapping.physicalName"":""salaryAfterTax""}}]})",null
null,null,"List(3, 7, List(deletionVectors, columnMapping), List(deletionVectors, columnMapping, appendOnly, invariants))"


In [0]:
%sql
describe history delta.`/Volumes/my_catalog/my_schema/my_volume/delta_four/`;

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
3,2025-08-15T13:14:26.000Z,78362573187593,phyominnthwin@gmail.com,OPTIMIZE,"Map(predicate -> [], auto -> true, clusterBy -> [], zOrderBy -> [], batchId -> 0)",null,null,0815-125944-m043vv0y-v2n,2,SnapshotIsolation,false,"Map(numRemovedFiles -> 3, numRemovedBytes -> 3660, p25FileSize -> 1330, numDeletionVectorsRemoved -> 1, minFileSize -> 1330, numAddedFiles -> 1, maxFileSize -> 1330, p75FileSize -> 1330, p50FileSize -> 1330, numAddedBytes -> 1330)",null,Databricks-Runtime/17.0.x-aarch64-photon-scala2.13
2,2025-08-15T13:14:23.000Z,78362573187593,phyominnthwin@gmail.com,UPDATE,"Map(predicate -> [""(id#10844 = 2)""])",null,null,0815-125944-m043vv0y-v2n,1,WriteSerializable,false,"Map(numRemovedFiles -> 0, numRemovedBytes -> 0, numCopiedRows -> 0, numDeletionVectorsAdded -> 1, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 3329, numDeletionVectorsUpdated -> 0, scanTimeMs -> 1822, numAddedFiles -> 1, numUpdatedRows -> 1, numAddedBytes -> 1216, rewriteTimeMs -> 1462)",null,Databricks-Runtime/17.0.x-aarch64-photon-scala2.13
1,2025-08-15T13:11:29.000Z,78362573187593,phyominnthwin@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,null,0815-125944-m043vv0y-v2n,0,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 2, numOutputBytes -> 1171)",null,Databricks-Runtime/17.0.x-aarch64-photon-scala2.13
0,2025-08-15T13:09:39.000Z,78362573187593,phyominnthwin@gmail.com,WRITE,"Map(mode -> ErrorIfExists, statsOnLoad -> false, partitionBy -> [])",null,null,0815-125944-m043vv0y-v2n,null,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 3, numOutputBytes -> 1273)",null,Databricks-Runtime/17.0.x-aarch64-photon-scala2.13


#### Delta UPSERT

In [0]:
data = [
  (5, "Mike", 120000, 108000),
  (6, "John", 150000, 135000),
  (7, "Sue", 100000, 90000),
  (8, "Mary", 120000, 108000)
]
df = spark.createDataFrame(data, ["id", "name", "salary", "salaryAfterTax"])


In [0]:
dlt_obj = DeltaTable.forPath(spark, "/Volumes/my_catalog/my_schema/my_volume/delta_four/")

dlt_obj.alias("dst").merge(df.alias("src"), "dst.id = src.id")\
        .whenMatchedUpdateAll()\
        .whenNotMatchedInsertAll()\
        .execute()

DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

In [0]:
%sql
select * from delta.`/Volumes/my_catalog/my_schema/my_volume/delta_four/`;

id,name,salary,salaryAfterTax
2,Jane,84000.0,108000.0
1,John,100000.0,90000.0
3,Joe,150000.0,135000.0
4,Mary,100000.0,null
5,Mike,120000.0,108000.0
6,John,150000.0,135000.0
7,Sue,100000.0,90000.0


#### Delta Schema Changes

In [0]:
%sql
ALTER TABLE delta.`/Volumes/my_catalog/my_schema/my_volume/delta_four/` SET TBLPROPERTIES ('delta.columnMapping.mode' = 'name')

In [0]:
%sql
alter table delta.`/Volumes/my_catalog/my_schema/my_volume/delta_four/` 
rename column name to employee_name;

#### Table Utility Command

In [0]:
%sql
describe delta.`/Volumes/my_catalog/my_schema/my_volume/delta_four/`;

col_name,data_type,comment
id,int,null
employee_name,string,null
salary,double,null
salaryAfterTax,double,null


In [0]:
%sql
describe detail delta.`/Volumes/my_catalog/my_schema/my_volume/delta_four/`;

format,id,name,description,location,createdAt,lastModified,partitionColumns,clusteringColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion,tableFeatures,statistics,clusterByAuto
delta,66814450-ee69-4751-9f86-07d4ba349df6,null,null,dbfs:/Volumes/my_catalog/my_schema/my_volume/delta_four,2025-08-15T13:09:37.467Z,2025-08-15T13:30:26.000Z,List(),List(),1,1401,"Map(delta.enableDeletionVectors -> true, delta.columnMapping.mode -> name, delta.columnMapping.maxColumnId -> 4)",3,7,"List(appendOnly, columnMapping, deletionVectors, invariants)","Map(numRowsDeletedByDeletionVectors -> 0, numDeletionVectors -> 0)",false


In [0]:
%sql
describe extended delta.`/Volumes/my_catalog/my_schema/my_volume/delta_four/`;

col_name,data_type,comment
id,int,null
employee_name,string,null
salary,double,null
salaryAfterTax,double,null
,,
# Delta Statistics Columns,,
Column Names,"id, name, salary, salaryAfterTax",
Column Selection Method,first-32,
,,
# Detailed Table Information,,


#### Data Versioning

In [0]:
%sql
describe history delta.`/Volumes/my_catalog/my_schema/my_volume/delta_four/`;

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
8,2025-08-15T13:34:38.000Z,78362573187593,phyominnthwin@gmail.com,RESTORE,"Map(version -> 6, timestamp -> null)",null,null,0815-125944-m043vv0y-v2n,7,Serializable,false,"Map(numRestoredFiles -> 0, removedFilesSize -> 0, numRemovedFiles -> 0, restoredFilesSize -> 0, numOfFilesAfterRestore -> 1, tableSizeAfterRestore -> 1401)",null,Databricks-Runtime/17.0.x-aarch64-photon-scala2.13
7,2025-08-15T13:30:26.000Z,78362573187593,phyominnthwin@gmail.com,RENAME COLUMN,"Map(oldColumnPath -> name, newColumnPath -> employee_name)",null,null,0815-125944-m043vv0y-v2n,6,WriteSerializable,true,Map(),null,Databricks-Runtime/17.0.x-aarch64-photon-scala2.13
6,2025-08-15T13:30:21.000Z,78362573187593,phyominnthwin@gmail.com,SET TBLPROPERTIES,"Map(properties -> {""delta.columnMapping.mode"":""name""})",null,null,0815-125944-m043vv0y-v2n,5,WriteSerializable,true,Map(),null,Databricks-Runtime/17.0.x-aarch64-photon-scala2.13
5,2025-08-15T13:28:25.000Z,78362573187593,phyominnthwin@gmail.com,OPTIMIZE,"Map(predicate -> [], auto -> true, clusterBy -> [], zOrderBy -> [], batchId -> 0)",null,null,0815-125944-m043vv0y-v2n,4,SnapshotIsolation,false,"Map(numRemovedFiles -> 4, numRemovedBytes -> 4973, p25FileSize -> 1401, numDeletionVectorsRemoved -> 1, minFileSize -> 1401, numAddedFiles -> 1, maxFileSize -> 1401, p75FileSize -> 1401, p50FileSize -> 1401, numAddedBytes -> 1401)",null,Databricks-Runtime/17.0.x-aarch64-photon-scala2.13
4,2025-08-15T13:28:23.000Z,78362573187593,phyominnthwin@gmail.com,MERGE,"Map(predicate -> [""(cast(id#11472 as bigint) = id#11476L)""], clusterBy -> [], matchedPredicates -> [{""actionType"":""update""}], statsOnLoad -> false, notMatchedBySourcePredicates -> [], notMatchedPredicates -> [{""actionType"":""insert""}])",null,null,0815-125944-m043vv0y-v2n,3,WriteSerializable,false,"Map(numTargetRowsCopied -> 0, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 3, numTargetBytesAdded -> 3643, numTargetBytesRemoved -> 0, numTargetDeletionVectorsAdded -> 1, numTargetRowsMatchedUpdated -> 1, executionTimeMs -> 3264, materializeSourceTimeMs -> 271, numTargetRowsInserted -> 2, numTargetRowsMatchedDeleted -> 0, numTargetDeletionVectorsUpdated -> 0, scanTimeMs -> 1378, numTargetRowsUpdated -> 1, numOutputRows -> 3, numTargetDeletionVectorsRemoved -> 0, numTargetRowsNotMatchedBySourceUpdated -> 0, numTargetChangeFilesAdded -> 0, numSourceRows -> 3, numTargetFilesRemoved -> 0, numTargetRowsNotMatchedBySourceDeleted -> 0, rewriteTimeMs -> 1522)",null,Databricks-Runtime/17.0.x-aarch64-photon-scala2.13
3,2025-08-15T13:14:26.000Z,78362573187593,phyominnthwin@gmail.com,OPTIMIZE,"Map(predicate -> [], auto -> true, clusterBy -> [], zOrderBy -> [], batchId -> 0)",null,null,0815-125944-m043vv0y-v2n,2,SnapshotIsolation,false,"Map(numRemovedFiles -> 3, numRemovedBytes -> 3660, p25FileSize -> 1330, numDeletionVectorsRemoved -> 1, minFileSize -> 1330, numAddedFiles -> 1, maxFileSize -> 1330, p75FileSize -> 1330, p50FileSize -> 1330, numAddedBytes -> 1330)",null,Databricks-Runtime/17.0.x-aarch64-photon-scala2.13
2,2025-08-15T13:14:23.000Z,78362573187593,phyominnthwin@gmail.com,UPDATE,"Map(predicate -> [""(id#10844 = 2)""])",null,null,0815-125944-m043vv0y-v2n,1,WriteSerializable,false,"Map(numRemovedFiles -> 0, numRemovedBytes -> 0, numCopiedRows -> 0, numDeletionVectorsAdded -> 1, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 3329, numDeletionVectorsUpdated -> 0, scanTimeMs -> 1822, numAddedFiles -> 1, numUpdatedRows -> 1, numAddedBytes -> 1216, rewriteTimeMs -> 1462)",null,Databricks-Runtime/17.0.x-aarch64-photon-scala2.13
1,2025-08-15T13:11:29.000Z,78362573187593,phyominnthwin@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,null,0815-125944-m043vv0y-v2n,0,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 2, numOutput

#### Time Traveling

In [0]:
%sql
restore table delta.`/Volumes/my_catalog/my_schema/my_volume/delta_four/` 
version as of 6;

table_size_after_restore,num_of_files_after_restore,num_removed_files,num_restored_files,removed_files_size,restored_files_size
1401,1,0,0,0,0


#### Table Properties

In [0]:
%sql
show tblproperties delta.`/Volumes/my_catalog/my_schema/my_volume/delta_four/`;

key,value
delta.columnMapping.maxColumnId,4
delta.columnMapping.mode,name
delta.enableDeletionVectors,true
delta.feature.appendOnly,supported
delta.feature.columnMapping,supported
delta.feature.deletionVectors,supported
delta.feature.invariants,supported
delta.minReaderVersion,3
delta.minWriterVersion,7


#### Shallow Cloning

In [0]:
%sql
create table my_catalog.my_schema.delta_fourv2
shallow clone my_catalog.my_schema.delta_four;

source_table_size,source_num_of_files,num_of_synced_transactions,num_removed_files,num_copied_files,removed_files_size,copied_files_size
1248,1,null,0,0,0,0


#### Deep Cloning

In [0]:
%sql
create table my_catalog.my_schema.delta_fourv3
clone delta.`/Volumes/my_catalog/my_schema/my_volume/delta_four/` version as of 3;

source_table_size,source_num_of_files,num_of_synced_transactions,num_removed_files,num_copied_files,removed_files_size,copied_files_size
1330,1,null,0,1,0,1330


#### Change Data Feed

In [0]:
%sql
alter table my_catalog.my_schema.delta_four
set tblproperties ('delta.enableChangeDataFeed' = 'true');

In [0]:
%sql
delete from my_catalog.my_schema.delta_four where id = 3;

num_affected_rows
1


In [0]:
%sql
select * from table_changes("my_catalog.my_schema.delta_four", 4);

id,name,salary,salaryAfterTax,_change_type,_commit_version,_commit_timestamp
4,Mary,100000.0,90000.0,insert,4,2025-08-15T13:43:04.000Z
5,Mike,120000.0,108000.0,insert,4,2025-08-15T13:43:04.000Z
3,Joe,150000.0,135000.0,delete,5,2025-08-15T13:44:13.000Z


#### Optimization

In [0]:
%sql
optimize delta.`/Volumes/my_catalog/my_schema/my_volume/delta_four/`;

path,metrics
dbfs:/Volumes/my_catalog/my_schema/my_volume/delta_four,"List(0, 0, List(null, null, 0.0, 0, 0), List(null, null, 0.0, 0, 0), 0, null, null, 0, 0, 1, 1, true, 0, 0, 1755265566404, 1755265567896, 8, 0, null, List(0, 0), null, 4, 4, 0, 0, null)"


#### Zordering

In [0]:
%sql
optimize delta.`/Volumes/my_catalog/my_schema/my_volume/delta_four/`
zorder by (id);

path,metrics
dbfs:/Volumes/my_catalog/my_schema/my_volume/delta_four,"List(0, 0, List(null, null, 0.0, 0, 0), List(null, null, 0.0, 0, 0), 0, List(minCubeSize(107374182400), List(0, 0), List(1, 1401), 0, List(0, 0), 0, null), null, 0, 0, 1, 1, false, 0, 0, 1755265642921, 1755265644383, 8, 0, null, List(0, 0), null, 4, 4, 0, 0, null)"


#### Liquid Clustering

In [0]:
%sql
alter table my_catalog.my_schema.delta_four
cluster by auto;